## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-28-07-36-53 +0000,bbc,Sanctions reimposed on Iran 10 years after lan...,https://www.bbc.com/news/articles/crme84w3n23o...
1,2025-09-28-07-26-49 +0000,bbc,Government to guarantee £1.5bn Jaguar Land Rov...,https://www.bbc.com/news/articles/cgl15ykerlro...
2,2025-09-28-07-12-34 +0000,bbc,"Kyiv hit by 'massive' Russian attack, mayor says",https://www.bbc.com/news/articles/c75qeqr5905o...
3,2025-09-28-07-01-28 +0000,nypost,Price tag of Obama’s presidential center repor...,https://nypost.com/2025/09/28/us-news/what-is-...
4,2025-09-28-06-32-38 +0000,nyt,Gunman Fires on Riverside Bar in North Carolin...,https://www.nytimes.com/2025/09/28/us/north-ca...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2457/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
192,trump,37
199,up,8
40,new,7
191,president,7
115,dies,6


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
24,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...,83
70,2025-09-27-21-15-59 +0000,wapo,"Trump calls for troops in Portland, escalating...",https://www.washingtonpost.com/politics/2025/0...,67
57,2025-09-27-22-19-00 +0000,wsj,"Trump Says He Is Ordering Troops to Portland, ...",https://www.wsj.com/us-news/trump-says-he-is-o...,65
48,2025-09-27-23-18-16 +0000,nyt,"Trump Says He Has Ordered Troops to Portland, ...",https://www.nytimes.com/2025/09/27/us/politics...,63
64,2025-09-27-21-42-04 +0000,bbc,Trump authorises 'full force' as Portland beco...,https://www.bbc.com/news/articles/cddmn6ge6e2o...,63


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
24,83,2025-09-28-02-26-00 +0000,wsj,President Trump has agreed to meet with the fo...,https://www.wsj.com/politics/policy/trump-to-h...
166,39,2025-09-27-09-30-00 +0000,wsj,A new barrier for foreign workers could open s...,https://www.wsj.com/economy/jobs/trump-h1b-fee...
70,30,2025-09-27-21-15-59 +0000,wapo,"Trump calls for troops in Portland, escalating...",https://www.washingtonpost.com/politics/2025/0...
13,28,2025-09-28-04-26-42 +0000,nypost,"Texas special ed teacher, ex-clown drops dead...",https://nypost.com/2025/09/28/us-news/texas-sp...
89,28,2025-09-27-18-50-03 +0000,bbc,FBI fires agents pictured kneeling at George F...,https://www.bbc.com/news/articles/cr4q5xrz9dpo...
68,27,2025-09-27-21-27-07 +0000,nypost,Anti-ICE protesters led by paid activists marc...,https://nypost.com/2025/09/27/us-news/brooklyn...
72,26,2025-09-27-21-03-57 +0000,nyt,Stampede at Actor’s Political Rally Kills at L...,https://www.nytimes.com/2025/09/27/world/asia/...
90,26,2025-09-27-18-29-53 +0000,nypost,Israel strikes 120 targets in Gaza Strip — inc...,https://nypost.com/2025/09/27/world-news/israe...
18,23,2025-09-28-03-29-21 +0000,nypost,Rep. Marjorie Taylor Greene warns in X rant t...,https://nypost.com/2025/09/27/us-news/rep-marj...
20,21,2025-09-28-03-05-31 +0000,nypost,"3 dead, 8 injured after gunman on boat opens f...",https://nypost.com/2025/09/27/us-news/north-ca...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
